Main interface for application:

In [1]:
from __future__ import print_function
from PIL import Image
from PIL import ImageTk
import tkinter as tki
from tkinter import messagebox
import threading
import datetime
import imutils
import cv2
import os
import time
import numpy as np
import uuid

import tensorflow as tf
from tensorflow import keras
from keras.models import Model

TF configuration for Nvidia Jetson nano:

In [2]:
#device = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(device[0], True)
#tf.config.experimental.set_virtual_device_configuration(device[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

In [3]:
global dirname
dirname = os.path.abspath('')

Initialize camera

In [4]:
global vs
vs = cv2.VideoCapture(0, cv2.CAP_DSHOW)

In [5]:
class MNN:
    def __init__(self, vs, model, dim):
        self.vs = vs
        self.dim = dim
        self.model = model
        self.frame = None
        self.thread = None
        self.stopEvent = None
        ret, frame = self.vs.read()
        self.panel = None
        self.isImage = False
        #Classes:
        self.class_names = ['aspergillus', 'bacillus_anthracis', 'bordetella_pertussis', 'cryptococcus', 'neisseria_gonorrhoeae', 'rhizopus_nigricans', 'rhizopus_stolonifer']
        
        #Type of microorganism:
        self.class_type = ['fungi', 'bacteria', 'bacteria', 'fungi', 'bacteria', 'fungi', 'fungi']
        self.vLoop = True
        
        self.root = tki.Tk()

        self.root.title('Microorganism NN')
        self.root.iconbitmap('1.ico')
        self.root.resizable(False, False)
        
        self.WF = tki.Frame(width=780, height=500)

    
        lbl = tki.Label(text="Preview")
        lbl.grid(sticky="w", pady=4, padx=5)

        blank_imageZ = np.zeros((96,128,3), np.uint8)  
        blank_image = cv2.cvtColor(blank_imageZ, cv2.COLOR_BGR2RGB)
        blank_image = Image.fromarray(blank_image)
        blank_image = ImageTk.PhotoImage(blank_image)
        
        self.ss = tki.Label(image=blank_image)
        self.ss.image = blank_image
        self.ss.grid(row=1, column=3, columnspan=2, padx=5, sticky="nesw")
        
        #Test prediction for loading all nessesary libraries
        self.model_predict(blank_imageZ)
        
        
        #UI
        label1 = tki.Label(text="Name:")
        label1.grid(row=2, column=3, pady=4, padx=4, sticky="ne")

        label2 = tki.Label(text="Type:")
        label2.grid(row=3, column=3, pady=4, padx=4, sticky="ne")

        label3 = tki.Label(text="Score:")
        label3.grid(row=4, column=3, pady=4, padx=4, sticky="ne")
        
        self.bName = tki.Entry()
        self.bName.grid(row=2, column=4, pady=4, padx=4, sticky="ne")

        self.tName = tki.Label(text="---")
        self.tName.grid(row=3, column=4, pady=4, padx=4, sticky="ne")
        
        self.scr = tki.Label(text="---")
        self.scr.grid(row=4, column=4, pady=4, padx=4, sticky="ne")
        
        hbtn = tki.Button(text="Predict", height=5, command=self.predict)
        hbtn.grid(row=5, column=0, padx=5, columnspan=2, sticky="nesw")

        obtn = tki.Button(text="Save", height=5, command=self.save)
        obtn.grid(row=5, column=3, columnspan=2, sticky="nesw")
        
        #New thread for openCV camera stream
        self.stopEvent = threading.Event()
        self.thread = threading.Thread(target=self.videoLoop)
        self.thread.start()
        
    def close(self):
        #Stop videoLoop on close windows
        self.vLoop = False
        
    def videoLoop(self):

        while self.vLoop:

            ret, frame = self.vs.read()
            self.frame = frame
            #resize camera feed to match UI size
            self.frameR = imutils.resize(self.frame, width=640)
		
            #Update UI
            image = cv2.cvtColor(self.frameR, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)
            image = ImageTk.PhotoImage(image)
		
            if self.panel is None:
                self.panel = tki.Label(image=image)
                self.panel.image = image
                self.panel.grid(row=1, column=0, columnspan=2, rowspan=4, padx=5, sticky='ewns')
            else:
                self.panel.configure(image=image)
                self.panel.image = image
    
    
        if(self.vLoop == False):
                self.root.destroy()
                sys.exit()
                
                
                
    #predict value with selected model
    def model_predict(self, img):
        
        #Set dimension of image for specified neural netowrk model
        if(self.dim == 's'):
            frameNN = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        else:
            frameNN = cv2.resize(img, dsize=(128, 96), interpolation=cv2.INTER_CUBIC)
        image = cv2.cvtColor(frameNN, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        
        #Update UI
        self.ss.configure(image=image)
        self.ss.image = image
        
        #Preprocess image for neural netowrk
        img_array = keras.preprocessing.image.img_to_array(frameNN)
        img_array = tf.expand_dims(img_array, 0)

        predictions = self.model.predict(img_array)
        return predictions
    
    
    def predict(self):
        self.isImage = True
        self.Fsave = self.frame;
        
        #predict
        predictions = self.model_predict(self.frame)
        score = tf.nn.softmax(predictions[0])
        
        
        #Update UI type and prediction
        self.bName.delete(0,"end")
        self.bName.insert(0,self.class_names[np.argmax(score)])
        
        self.tName.config(text=self.class_type[np.argmax(score)])
        
        prc = 100 * np.max(score)
        self.scr.config(text=prc)

    #save image
    def save(self):
        if(not self.isImage):
            messagebox.showinfo("Error", "You need to first predict image to save it or create new data")
        else:
            infoStr = "Image with class {} saved.".format(self.bName.get())
            filename = "{}.{}.jpg".format(self.bName.get(),uuid.uuid4())
            cv2.imwrite(("saved/"+filename).format(self.bName.get(),uuid.uuid4()),self.Fsave)
            messagebox.showinfo("Success", infoStr)

Window with selecting model for microorganism recognation

In [6]:
class SelectWindow:
    def __init__(self):

        self.root = tki.Tk()
        self.root.title('Microorganism NN - select model')
        self.root.iconbitmap('1.ico')
        
        label = tki.Label(text="Select model:")
        label.grid(row=1, column=1, pady=4, padx=4, sticky="ne")
        
        self.cBtn = tki.Button(text="Custom model", height=5, command=self.custom)
        self.cBtn.grid(row=2, column=1, padx=5, columnspan=2, sticky="nesw")
        
        self.mBtn = tki.Button(text="MobilenetV3 - transfer learning", height=5, command=self.inceptionV3)
        self.mBtn.grid(row=3, column=1, padx=5, columnspan=2, sticky="nesw")
        


    def custom(self):
        self.root.destroy()
        self.sModel = keras.models.load_model(os.path.join(dirname, 'model'));
        self.Type = 'r'
        self.start()
        
    def inceptionV3(self):
        self.root.destroy()
        self.sModel = keras.models.load_model(os.path.join(dirname, 'model_mobilenetV3'));
        self.Type = 's'
        self.start()
        
    def start(self):
        NN = MNN(vs, self.sModel, self.Type)
        NN.root.protocol("WM_DELETE_WINDOW", NN.close)
        NN.root.mainloop()


It can take some time to first time load model

In [7]:
select = SelectWindow()
select.root.mainloop()